<a href="https://colab.research.google.com/github/affandhanbad-dev/DL_Learning_Colab_Notebook/blob/main/DL_IMDB_sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle

In [ ]:
import os
import json
from zipfile import ZipFile
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
kaggle_dictionary = json.load(open("kaggle.json"))

In [ ]:
os.environ["KAGGLE_USERNAME"] = kaggle_dictionary["username"]
os.environ["KAGGLE_KEY"] = kaggle_dictionary["key"]

In [ ]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
License(s): other
  0% 0.00/25.7M [00:00<?, ?B/s]
100% 25.7M/25.7M [00:00<00:00, 1.58GB/s]


In [ ]:
!ls

imdb-dataset-of-50k-movie-reviews.zip  kaggle.json  sample_data


In [ ]:
with ZipFile("imdb-dataset-of-50k-movie-reviews.zip", "r") as zip_ref:
    zip_ref.extractall()

In [ ]:
data = pd.read_csv("IMDB Dataset.csv")

In [ ]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [ ]:
data['sentiment'].value_counts()

,count
sentiment,
positive,25000
negative,25000


In [ ]:
data.replace({"sentiment": {"positive": 1, "negative": 0}},inplace=True)

/tmp/ipython-input-310373736.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data.replace({"sentiment": {"positive": 1, "negative": 0}},inplace=True)


In [ ]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [ ]:
train_data, test_data = train_test_split(data, test_size=0.2)

In [ ]:
tokenizer = Tokenizer(num_words = 5000)
tokenizer.fit_on_texts(train_data['review'])
X_train = pad_sequences(tokenizer.texts_to_sequences(train_data['review']),maxlen =200)
X_test = pad_sequences(tokenizer.texts_to_sequences(test_data['review']),maxlen =200)

In [ ]:
X_train.shape

(40000, 200)

In [ ]:
X_train

array([[   0,    0,    0, ..., 1632,    6,  150],
       [   1,   19,  724, ...,  161,   18,  318],
       [ 926,   46,    6, ...,  552,   15, 4656],
       ...,
       [2417,   23, 4307, ..., 1147,   15,   93],
       [   0,  805, 1980, ..., 4909,   49, 1095],
       [   1,   17,   11, ..., 2757,   11,  909]], dtype=int32)

In [ ]:
Y_train = train_data['sentiment']
Y_test = test_data['sentiment']

In [ ]:
Y_train.shape

(40000,)

In [ ]:
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=128, input_length=200))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation="sigmoid"))

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
model.fit(X_train, Y_train, batch_size=128, epochs=3, validation_data=(X_test, Y_test))

Epoch 1/3
313/313 ━━━━━━━━━━━━━━━━━━━━ 205s 652ms/step - accuracy: 0.7748 - loss: 0.4832 - val_accuracy: 0.8606 - val_loss: 0.3311
Epoch 2/3
313/313 ━━━━━━━━━━━━━━━━━━━━ 200s 638ms/step - accuracy: 0.8666 - loss: 0.3271 - val_accuracy: 0.8447 - val_loss: 0.3510
Epoch 3/3
313/313 ━━━━━━━━━━━━━━━━━━━━ 205s 648ms/step - accuracy: 0.8888 - loss: 0.2846 - val_accuracy: 0.8785 - val_loss: 0.3031


In [ ]:
loss, accuracy = model.evaluate(X_test, Y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

313/313 ━━━━━━━━━━━━━━━━━━━━ 42s 134ms/step - accuracy: 0.8797 - loss: 0.3021
Test Loss: 0.3030523955821991
Test Accuracy: 0.8784999847412109


In [ ]:
def predict_sentiment(review):
  sequence = tokenizer.texts_to_sequences([review])
  padded_sequence = pad_sequences(sequence, maxlen=200)
  prediction = model.predict(padded_sequence)
  sentiment = "positive" if prediction[0][0] > 0.5 else "negative"
  return sentiment

In [ ]:
# new_review = "This movie was fantastic. I loved it."
# sentiment = predict_sentiment(new_review)
# print(f"The sentiment of the review is: {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 440ms/step
The sentiment of the review is: positive


In [ ]:
import pickle
with open("IMDB_sentiment_model.pkl", "wb") as file:
    pickle.dump(model, file)

In [ ]:
# new_review = "the worst movie ever"
# sentiment = predict_sentiment(new_review)
# print(f"The sentiment of the review is: {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step
The sentiment of the review is: negative


In [ ]:
new_review =input("please enter your review")
sentiment = predict_sentiment(new_review)
print(f"The sentiment of the review is: {sentiment}")

please enter your reviewgood movie highly recomend
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step
The sentiment of the review is: positive
